In [1]:
import pandas as pd
import sqlite3

In [2]:
input_df = pd.read_csv('https://raw.githubusercontent.com/andandandand/images-for-colab-notebooks/main/DM_SALES.csv', sep=';')
input_df

,ORDER_ID_PSEUDONYM,SUM(QUANTITY),YEAR(ORDER_DATE),State,Price
0,1,2,2018,Hawaii,$44
1,2,3,2019,California,$43
2,3,4,2018,Massachusetts,$84
3,4,5,2019,New York,$15
4,5,7,2019,Colorado,$35
...,...,...,...,...,...
67,21,5,2019,NaN,NaN
68,22,7,2019,NaN,NaN
69,23,8,2019,NaN,NaN
70,24,1,2019,NaN,NaN


In [3]:
# the parehtheses need to be removed from the column names
clean_column_names = [name.replace('(','_').replace(')','') for name in list(input_df.columns)]
input_df_2 = pd.DataFrame(input_df.values, columns=clean_column_names)
input_df_2

,ORDER_ID_PSEUDONYM,SUM_QUANTITY,YEAR_ORDER_DATE,State,Price
0,1,2,2018,Hawaii,$44
1,2,3,2019,California,$43
2,3,4,2018,Massachusetts,$84
3,4,5,2019,New York,$15
4,5,7,2019,Colorado,$35
...,...,...,...,...,...
67,21,5,2019,NaN,NaN
68,22,7,2019,NaN,NaN
69,23,8,2019,NaN,NaN
70,24,1,2019,NaN,NaN


## SQL Hepler Functions

In [4]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table


    
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

## Execute Query

minput_df.values

In [5]:
# Step 2: Upload the dataframe to a SQL Table

pd_to_sqlDB(input_df_2,table_name='DM_SALES', db_name='default.db')


2021-11-12 14:44:36 INFO: SQL DB default.db created
2021-11-12 14:44:36 INFO: SQL Table DM_SALES created with 5 columns
2021-11-12 14:44:36 INFO: 72 rows uploaded to DM_SALES


In [7]:
# Step 3: Write the SQL query in a string variable
sql_query_string = """
    SELECT ORDER_ID_PSEUDONYM, SUM_QUANTITY as QTY
    FROM DM_SALES
    WHERE YEAR_ORDER_DATE=2019 
    GROUP BY ORDER_ID_PSEUDONYM
    HAVING SUM_QUANTITY=1
"""
 
# Step 4: Execute the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,ORDER_ID_PSEUDONYM,QTY
0,8,1
1,16,1
2,24,1
3,39,1
4,47,1
